In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import data

In [2]:
#btc options data
df=pd.read_csv('data/btc_data.csv', sep=",", index_col=0)

#df.drop(columns=['Unnamed: 0', 'interest_rate'], inplace=True)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1029 entries, 0 to 1029
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   best_bid_amount     1029 non-null   float64
 1   best_ask_amount     1029 non-null   float64
 2   bid_iv              1029 non-null   float64
 3   ask_iv              1029 non-null   float64
 4   underlying_index    1029 non-null   object 
 5   underlying_price    1029 non-null   float64
 6   mark_iv             1029 non-null   float64
 7   best_bid_price      1029 non-null   float64
 8   best_ask_price      1029 non-null   float64
 9   open_interest       1029 non-null   float64
 10  max_price           1029 non-null   float64
 11  min_price           1029 non-null   float64
 12  last_price          864 non-null    float64
 13  asks                1029 non-null   object 
 14  bids                1029 non-null   object 
 15  settlement_price    989 non-null    float64
 16  instrument_